In [ ]:
from aoc import *
import re
import os
from aocd.models import Puzzle as AOCDPuzzle

def Puzzle(day, year=2023):
   return AOCDPuzzle(year=year, day=day)

In [ ]:
p = Puzzle(day=1)

In [ ]:
WORDS = {"one": 1, "two": 2, "three": 3, "four": 4, "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9}

def words_to_digits(s: str):
    while True:
        first = 10**9
        word = None
        digit = None
        for w, d in WORDS.items():
            idx = s.find(w)
            if idx >= 0 and idx < first:
                first, word, digit = idx, w, d
        if word is None:
            break
        s = s.replace(word, str(digit), 1)
    return s

def trebuchet(x):
    def digits(line):
        return [int(d) for d in line if d.isdigit()]
    lines = [digits(line) for line in x.splitlines()]
    return sum(digits[0] * 10 + digits[-1] for digits in lines)

assert trebuchet(p.examples[0].input_data) == 142
assert trebuchet(words_to_digits('''two1nine
eightwothree
abcone2threexyz
xtwone3four
4nineeightseven2
zoneight234
7pqrstsixteen''')) == 281

In [ ]:
print (trebuchet(p.input_data), trebuchet(words_to_digits(p.input_data)))

In [ ]:
p = Puzzle(day=2)

In [ ]:
def parse_cube_game(line):
    def parse_round(r):
        return dict((tpl[1], int(tpl[0]))
            for tpl in [
                tuple(x for x in t.split(' '))
                for t in r.split(', ')
            ])
    game, rest = line.strip().split(': ')
    game = int(game[5:])
    rounds = [parse_round(r)
        for r in rest.split('; ')
    ]
    return game, rounds

def valid_round(round: dict, cubes: dict):
    return all(balls <= cubes[color] for color, balls in round.items())

def cube_game(data, cubes: dict = {'red': 12, 'green': 13, 'blue': 14}):
    valid_games = 0
    total_power = 0
    for line in data.splitlines():
        game, rounds = parse_cube_game(line)
        if all(valid_round(r, cubes) for r in rounds):
            valid_games += game
        min_balls = defaultdict(int)
        for r in rounds:
            for color, count in r.items():
                min_balls[color] = max(min_balls[color], count)
        power = min_balls['red'] * min_balls['green'] * min_balls['blue']
        total_power += power
    return valid_games, total_power

assert (8, 2286) == cube_game(p.examples[0].input_data)

In [ ]:
p.answers = cube_game(p.input_data)

In [ ]:
p = Puzzle(day=3)

In [ ]:
def parse_gears(input):
    total = 0
    gear_ratio = 0
    schematic = [l.strip() for l in input.splitlines()]
    gears = defaultdict(set)
    def is_part(row, start, end):
        result = False
        for x in range(start, end):
            for xx, yy in neighbors8((x, row)):
                if xx < 0 or xx >= len(line) or yy < 0 or yy >= len(schematic):
                    continue
                n = schematic[yy][xx]
                if n.isdigit() or n == '.':
                    continue
                if n == '*':
                    gears[(xx, yy)].add((row, start, int(schematic[row][start:end])))
                result = True
        return result
    for row, line in enumerate(schematic):
        for m in re.finditer(r'\d+', line):
            start, end = m.span()
            num = int(line[start:end])
            if is_part(row, start, end):
                total += num
    for pos, parts in gears.items():
        if len(parts) != 2:
            continue
        parts = list(parts)
        gear_ratio += parts[0][-1] * parts[1][-1]
    return total, gear_ratio

assert (4361, 467835) == parse_gears(p.examples[0].input_data)

In [ ]:
p.answers = parse_gears(p.input_data)

In [ ]:
p = Puzzle(day=4)

In [ ]:
def scratchcards(data):
    total = 0
    copies = defaultdict(int)
    for line in data.splitlines():
        m = re.match(r'^Card\s+(\d+): ([\d\s]+) \| ([\d\s]+)', line.strip())
        assert m, line
        card, winners, have = int(m.group(1)), set(vector(m.group(2))), set(vector(m.group(3)))
        hits = len(winners & have)
        points = 2 ** (hits - 1) if hits > 0 else 0
        total += points
    return total

assert 13 == scratchcards(p.examples[0].input_data)

In [ ]:
p.answer_a = scratchcards(p.input_data)